# Chinese HandWritten Numbers Classification

## Customized dataset

In [8]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

import os
import pandas as pd
from torchvision.io import read_image

class ChineseMINISTDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_tranform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx,1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

## Data preprocessing

In [9]:
from torchvision.transforms import ToTensor, Lambda

transform = Lambda(lambda y : y.float())
# transform the integer label to one-hot encoding vector
target_transform = Lambda(lambda y : torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y-1), value=1))

## Start to build a model

In [10]:
from torch import nn
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


### Define a NN class

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(8192, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
       
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output   

In [12]:
model = CNN()
print(model)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=8192, out_features=10, bias=True)
)


### Load dataset

In [13]:
training_path = 'training_set.csv'
validation_path = 'validation_set.csv'
testing_path = 'testing_set.csv'
final_train_path = 'final_training_set.csv'
img_dir = 'numbers'
training_datasets = ChineseMINISTDataset(training_path, img_dir, transform, target_transform)
validation_datasets = ChineseMINISTDataset(validation_path, img_dir, transform, target_transform)
testing_datasets = ChineseMINISTDataset(testing_path, img_dir, transform, target_transform)
final_training_datasets = ChineseMINISTDataset(final_train_path, img_dir, transform, target_transform)

In [14]:
train_dataloader = DataLoader(training_datasets, batch_size=32)
validate_dataloader = DataLoader(validation_datasets, batch_size=32)
test_dataloader = DataLoader(testing_datasets, batch_size=32)
final_train_dataloader = DataLoader(final_training_datasets, batch_size=32)

### set hyperparameters

In [15]:
learning_rate = 1e-2
batch_size = 32
epochs = 15

### choose loss function

In [16]:
loss_fn = nn.CrossEntropyLoss()

### choose optimizer

In [17]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-2)

### define train loop and valiadate loop

In [18]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
def validate_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    validate_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            validate_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    validate_loss /= num_batches
    correct /= size
    print(f"Validate Error: \n Accuracy : {(100*correct):>0.1f}%, Avg loss: {validate_loss:>8f} \n")

### train and test

In [19]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(final_train_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 6.565631  [    0/ 7998]
loss: 0.363906  [ 3200/ 7998]
loss: 1.078975  [ 6400/ 7998]
Epoch 2
-------------------------------
loss: 0.119232  [    0/ 7998]
loss: 0.051258  [ 3200/ 7998]
loss: 0.673055  [ 6400/ 7998]
Epoch 3
-------------------------------
loss: 0.055369  [    0/ 7998]
loss: 0.050678  [ 3200/ 7998]
loss: 0.634909  [ 6400/ 7998]
Epoch 4
-------------------------------
loss: 0.022679  [    0/ 7998]
loss: 0.082705  [ 3200/ 7998]
loss: 0.346044  [ 6400/ 7998]
Epoch 5
-------------------------------
loss: 0.038810  [    0/ 7998]
loss: 0.032603  [ 3200/ 7998]
loss: 0.252924  [ 6400/ 7998]
Epoch 6
-------------------------------
loss: 0.009729  [    0/ 7998]
loss: 0.122204  [ 3200/ 7998]
loss: 0.239973  [ 6400/ 7998]
Epoch 7
-------------------------------
loss: 0.013339  [    0/ 7998]
loss: 0.033820  [ 3200/ 7998]
loss: 0.164968  [ 6400/ 7998]
Epoch 8
-------------------------------
loss: 0.010436  [    0/ 7998]
loss: 0.003787  [ 32

In [20]:
test_loop(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 92.5%, Avg loss: 0.383298 



## Save model

In [21]:
torch.save(model.state_dict(), 'model_weights.pth')

### Done!